## Instrucciones de Uso

**Este notebook requiere datos procesados del notebook 03_encoding_transformaciones.ipynb**

### Flujo:
1. **Ejecutar primero** `03_encoding_transformaciones.ipynb` completamente
2. **Asegurarse** de que al final del notebook 03 se ejecute el código para guardar datos
3. **Luego ejecutar** este notebook que cargará automáticamente los datos balanceados con SMOTE

# Modelos Avanzados de Machine Learning
## Predicción de Consumo de Sustancias Psicoactivas - ENCSPA 2019

### Objetivo
Implementar y comparar modelos avanzados de Machine Learning para mejorar la predicción de consumo de marihuana, optimizando el rendimiento más allá de los modelos baseline.

### Modelos a Implementar
- **XGBoost**: Gradient boosting optimizado con regularización
- **LightGBM**: Gradient boosting rápido y eficiente en memoria
- **Random Forest**: Ensemble de árboles de decisión (baseline)
- **Gradient Boosting**: Boosting secuencial tradicional

### Metodología
- Validación cruzada estratificada (5-fold)
- Optimización de hiperparámetros con RandomizedSearchCV
- Métricas: Accuracy, Precision, Recall, F1, ROC-AUC
- Análisis de feature importance y tiempos de entrenamiento

## 1. Importación de Librerías y Configuración

In [3]:
# Librerías base
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import (
    StratifiedKFold, cross_validate, RandomizedSearchCV
)
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier
)
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, confusion_matrix, classification_report,
    roc_curve, auc
)

# Modelos avanzados
import xgboost as xgb
import lightgbm as lgb

# Utilidades
import time
import joblib
from datetime import datetime

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Librerías importadas exitosamente")

Librerías importadas exitosamente


## 2. Carga de Datos Preprocessados

In [4]:
print("Intentando cargar datos del notebook 03_encoding_transformaciones...")

try:
    # Cargar datos balanceados con SMOTE (para entrenamiento)
    X_train_balanced = pd.read_pickle('../data/processed/X_train_balanced.pkl')
    y_train_balanced = pd.read_pickle('../data/processed/y_train_balanced.pkl')
    
    # Cargar datos de test (SIN balanceo para evaluación realista)
    X_test_transformed = pd.read_pickle('../data/processed/X_test_transformed.pkl')
    y_test = pd.read_pickle('../data/processed/y_test.pkl')
    
    # Cargar nombres de features
    feature_names = pd.read_pickle('../data/processed/feature_names.pkl')
    
    # Usar datos balanceados para entrenamiento y originales para test
    X_train_final = X_train_balanced
    y_train_final = y_train_balanced
    X_test_final = X_test_transformed
    y_test_final = y_test
    
    print("Datos cargados desde notebook 03 (CON SMOTE para train)")
    print(f"   • Features disponibles: {len(feature_names)}")
    
except FileNotFoundError:
    print("Archivos del notebook 03 no encontrados")
    print("Ejecutando notebook 03 para generar datos procesados...")
    
    # Ejecutar notebook 03 si está en el mismo directorio
    try:
        get_ipython().run_line_magic('run', '03_encoding_transformaciones.ipynb')
        
        # Ahora deberían estar disponibles las variables
        X_train_final = X_train_balanced
        y_train_final = y_train_balanced
        X_test_final = X_test_transformed
        y_test_final = y_test
        
        print("Notebook 03 ejecutado exitosamente")
        
    except Exception as e:
        print(f"Error ejecutando notebook 03: {e}")
        print("Cargando datos con pipeline básico...")

        # FALLBACK: Pipeline básico si todo falla
        df = pd.read_csv('../data/g_capitulos.csv')
        
        # Variables según análisis previos
        target_var = 'G_11_F'
        categorical_features = ['G_01', 'G_02', 'G_03', 'G_04', 'G_05']
        numerical_features = [
            'G_06_A', 'G_06_B', 'G_06_C', 'G_06_D',
            'G_07', 'G_08_A', 'G_08_B', 'G_01_A', 'G_02_A'
        ]
        
        # Filtrar casos válidos
        df_clean = df[df[target_var].isin([1, 2])].copy()
        
        # Preparar variables
        features = categorical_features + numerical_features
        X = df_clean[features]
        y = (df_clean[target_var] == 1).astype(int)
        
        # Preprocessing básico
        from sklearn.model_selection import train_test_split
        from sklearn.impute import SimpleImputer
        from sklearn.preprocessing import LabelEncoder, StandardScaler
        
        # Train-test split
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )
        
        # Imputación
        cat_imputer = SimpleImputer(strategy='most_frequent')
        num_imputer = SimpleImputer(strategy='median')
        
        X_train[categorical_features] = cat_imputer.fit_transform(X_train[categorical_features])
        X_test[categorical_features] = cat_imputer.transform(X_test[categorical_features])
        
        X_train[numerical_features] = num_imputer.fit_transform(X_train[numerical_features])
        X_test[numerical_features] = num_imputer.transform(X_test[numerical_features])
        
        # Encoding
        le_dict = {}
        for col in categorical_features:
            le = LabelEncoder()
            X_train[col] = le.fit_transform(X_train[col].astype(str))
            X_test[col] = le.transform(X_test[col].astype(str))
            le_dict[col] = le
        
        # Escalamiento
        scaler = StandardScaler()
        X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])
        X_test[numerical_features] = scaler.transform(X_test[numerical_features])
        
        # Sin SMOTE en fallback
        X_train_final = X_train
        y_train_final = y_train
        X_test_final = X_test
        y_test_final = y_test
        
        print("Pipeline básico ejecutado (SIN SMOTE)")

# Verificación final de datos
print("\nVERIFICACIÓN FINAL DE DATOS:")
print(f"   • Train: {X_train_final.shape[0]:,} muestras, {X_train_final.shape[1]} features")
print(f"   • Test: {X_test_final.shape[0]:,} muestras, {X_test_final.shape[1]} features")
print(f"   • NaN en datos: {X_train_final.isnull().sum().sum() + X_test_final.isnull().sum().sum()}")

print(f"\nDISTRIBUCIÓN DE CLASES:")
print(f"   • Train - No consume: {(y_train_final == 0).sum():,} ({(y_train_final == 0).mean():.1%})")
print(f"   • Train - Sí consume: {(y_train_final == 1).sum():,} ({(y_train_final == 1).mean():.1%})")
print(f"   • Test - No consume: {(y_test_final == 0).sum():,} ({(y_test_final == 0).mean():.1%})")
print(f"   • Test - Sí consume: {(y_test_final == 1).sum():,} ({(y_test_final == 1).mean():.1%})")

# Verificar si tenemos balance (indicativo de SMOTE)
train_balance = (y_train_final == 1).mean()
if train_balance > 0.4:
    print(f"SMOTE aplicado - Balance train: {train_balance:.1%}")
else:
    print(f"Sin SMOTE - Balance original: {train_balance:.1%}")

Intentando cargar datos del notebook 03_encoding_transformaciones...
Datos cargados desde notebook 03 (CON SMOTE para train)
   • Features disponibles: 14

VERIFICACIÓN FINAL DE DATOS:
   • Train: 31,442 muestras, 14 features
   • Test: 4,686 muestras, 14 features
   • NaN en datos: 0

DISTRIBUCIÓN DE CLASES:
   • Train - No consume: 15,721 (50.0%)
   • Train - Sí consume: 15,721 (50.0%)
   • Test - No consume: 3,931 (83.9%)
   • Test - Sí consume: 755 (16.1%)
SMOTE aplicado - Balance train: 50.0%


## 3. Definición de Modelos y Espacios de Hiperparámetros

In [5]:
# Configuración de modelos avanzados
modelos_avanzados = {
    'XGBoost': {
        'model': xgb.XGBClassifier(
            random_state=42, 
            eval_metric='logloss',
            verbosity=0
        ),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5, 6],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0],
            'reg_alpha': [0, 0.1, 0.5],
            'reg_lambda': [1, 1.5, 2]
        }
    },
    
    'LightGBM': {
        'model': lgb.LGBMClassifier(
            random_state=42,
            verbose=-1,
            force_col_wise=True
        ),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5, 6],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0],
            'reg_alpha': [0, 0.1, 0.5],
            'reg_lambda': [0, 0.1, 0.5],
            'num_leaves': [15, 31, 50]
        }
    },
    
    'RandomForest': {
        'model': RandomForestClassifier(
            random_state=42,
            n_jobs=-1
        ),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [10, 15, 20, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2', 0.8],
            'bootstrap': [True, False]
        }
    },
    
    'GradientBoosting': {
        'model': GradientBoostingClassifier(
            random_state=42
        ),
        'params': {
            'n_estimators': [100, 200, 300],
            'max_depth': [3, 4, 5],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9, 1.0],
            'max_features': ['sqrt', 'log2', None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }
    }
}

print("Modelos configurados:")
for nombre, config in modelos_avanzados.items():
    n_params = len(config['params'])
    total_combinations = np.prod([len(v) for v in config['params'].values()])
    print(f"   • {nombre}: {n_params} hiperparámetros, {total_combinations:,} combinaciones posibles")

print(f"\nEstrategia de búsqueda:")
print(f"   • RandomizedSearchCV con 50 iteraciones por modelo")
print(f"   • Validación cruzada: StratifiedKFold con 5 folds")
print(f"   • Métrica de optimización: ROC-AUC")

Modelos configurados:
   • XGBoost: 7 hiperparámetros, 2,916 combinaciones posibles
   • LightGBM: 8 hiperparámetros, 8,748 combinaciones posibles
   • RandomForest: 6 hiperparámetros, 648 combinaciones posibles
   • GradientBoosting: 7 hiperparámetros, 2,187 combinaciones posibles

Estrategia de búsqueda:
   • RandomizedSearchCV con 50 iteraciones por modelo
   • Validación cruzada: StratifiedKFold con 5 folds
   • Métrica de optimización: ROC-AUC


## 4. Validación Cruzada Baseline

In [6]:
# Configuración de validación cruzada con semilla fija
RANDOM_SEED_CV = 42  # Semilla específica
cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED_CV)

# Métricas a evaluar
scoring_metrics = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

# Función para evaluar modelo con validación cruzada
def evaluar_modelo_cv(modelo, X, y, cv, scoring, nombre_modelo):
    """
    Evalúa un modelo usando validación cruzada y devuelve métricas detalladas
    """
    print(f"   Evaluando {nombre_modelo} (BASELINE - hiperparámetros por defecto)...")
    
    start_time = time.time()
    
    # Realizar validación cruzada
    cv_results = cross_validate(
        modelo, X, y, 
        cv=cv, 
        scoring=scoring,
        return_train_score=True,
        n_jobs=-1
    )
    
    training_time = time.time() - start_time
    
    # Organizar resultados
    resultados = {
        'modelo': nombre_modelo,
        'tiempo_entrenamiento': training_time
    }
    
    # Agregar métricas de test y train
    for metric in scoring.keys():
        test_scores = cv_results[f'test_{metric}']
        train_scores = cv_results[f'train_{metric}']
        
        resultados[f'{metric}_test_mean'] = test_scores.mean()
        resultados[f'{metric}_test_std'] = test_scores.std()
        resultados[f'{metric}_train_mean'] = train_scores.mean()
        resultados[f'{metric}_train_std'] = train_scores.std()
        resultados[f'{metric}_overfitting'] = train_scores.mean() - test_scores.mean()
    
    print(f"     Completado en {training_time:.2f}s, F1-CV (baseline): {resultados['f1_test_mean']:.4f}")
    
    return resultados

# Evaluar todos los modelos baseline (sin optimización)
print("EVALUACION BASELINE - MODELOS SIN OPTIMIZAR (FASE 1)")
print("=" * 70)
print("Objetivo: Establecer línea base de rendimiento con hiperparámetros por defecto")
print(f"Semilla CV: {RANDOM_SEED_CV}")
print("=" * 70)

resultados_baseline = []

for nombre, config in modelos_avanzados.items():
    modelo_base = config['model']
    resultado = evaluar_modelo_cv(
        modelo_base, X_train_final, y_train_final, 
        cv_strategy, scoring_metrics, nombre
    )
    resultados_baseline.append(resultado)

# Convertir a DataFrame para análisis
df_baseline = pd.DataFrame(resultados_baseline)

print(f"\nRESULTADOS BASELINE (Validación Cruzada 5-fold):")
print("=" * 80)
print("IMPORTANTE: Estos son resultados con hiperparámetros POR DEFECTO")
print("=" * 80)

# Mostrar métricas principales
metricas_principales = ['accuracy_test_mean', 'precision_test_mean', 'recall_test_mean', 
                       'f1_test_mean', 'roc_auc_test_mean', 'tiempo_entrenamiento']

df_display = df_baseline[['modelo'] + metricas_principales].copy()
for col in metricas_principales[:-1]:  # Exclude tiempo_entrenamiento
    df_display[col] = df_display[col].apply(lambda x: f"{x:.4f}")
df_display['tiempo_entrenamiento'] = df_display['tiempo_entrenamiento'].apply(lambda x: f"{x:.2f}s")

print(df_display.to_string(index=False))

# Identificar mejor modelo por F1-score
mejor_modelo_f1 = df_baseline.loc[df_baseline['f1_test_mean'].idxmax(), 'modelo']
mejor_f1_score = df_baseline['f1_test_mean'].max()

print(f"\nMEJOR MODELO BASELINE (F1-Score): {mejor_modelo_f1} con F1 = {mejor_f1_score:.4f}")

# Ranking baseline
print(f"\nRANKING BASELINE (por F1-Score):")
df_baseline_sorted = df_baseline.sort_values('f1_test_mean', ascending=False)
for i, (_, row) in enumerate(df_baseline_sorted.iterrows(), 1):
    print(f"   {i}. {row['modelo']:15s}: F1={row['f1_test_mean']:.4f}, AUC={row['roc_auc_test_mean']:.4f}")

EVALUACION BASELINE - MODELOS SIN OPTIMIZAR (FASE 1)
Objetivo: Establecer línea base de rendimiento con hiperparámetros por defecto
Semilla CV: 42
   Evaluando XGBoost (BASELINE - hiperparámetros por defecto)...
     Completado en 1.98s, F1-CV (baseline): 0.9233
   Evaluando LightGBM (BASELINE - hiperparámetros por defecto)...
     Completado en 3.07s, F1-CV (baseline): 0.9245
   Evaluando RandomForest (BASELINE - hiperparámetros por defecto)...
     Completado en 1.75s, F1-CV (baseline): 0.9227
   Evaluando GradientBoosting (BASELINE - hiperparámetros por defecto)...
     Completado en 2.27s, F1-CV (baseline): 0.9151

RESULTADOS BASELINE (Validación Cruzada 5-fold):
IMPORTANTE: Estos son resultados con hiperparámetros POR DEFECTO
          modelo accuracy_test_mean precision_test_mean recall_test_mean f1_test_mean roc_auc_test_mean tiempo_entrenamiento
         XGBoost             0.9217              0.9050           0.9423       0.9233            0.9764                1.98s
        L

## 5. Optimización de Hiperparámetros

In [7]:
# CONFIGURACION DE REPRODUCIBILIDAD Y OPTIMIZACION
RANDOM_SEED = 42    # Semilla
N_ITER_SEARCH = 50  # Número de iteraciones
CV_FOLDS = 3        # Folds para optimización (menor para rapidez)

# Configurar numpy para reproducibilidad
np.random.seed(RANDOM_SEED)

print(f"CONFIGURACION DE REPRODUCIBILIDAD:")
print(f"   Semilla global: {RANDOM_SEED}")
print(f"   Iteraciones búsqueda: {N_ITER_SEARCH}")
print(f"   Folds CV optimización: {CV_FOLDS}")
print("   Todas las ejecuciones futuras darán resultados idénticos")
print()

# Función para optimizar hiperparámetros
def optimizar_modelo(nombre_modelo, modelo_base, param_grid, X, y):
    """
    Optimiza hiperparámetros usando RandomizedSearchCV
    """
    print(f"   OPTIMIZANDO: {nombre_modelo}")
    print(f"   Búsqueda aleatoria con {N_ITER_SEARCH} iteraciones y {CV_FOLDS}-fold CV")
    
    # Configurar búsqueda aleatoria
    random_search = RandomizedSearchCV(
        estimator=modelo_base,
        param_distributions=param_grid,
        n_iter=N_ITER_SEARCH,
        cv=StratifiedKFold(n_splits=CV_FOLDS, shuffle=True, random_state=RANDOM_SEED),
        scoring='f1',  # Optimizar por F1-score
        n_jobs=-1,
        random_state=RANDOM_SEED,
        verbose=0
    )
    
    start_time = time.time()
    
    # Ejecutar búsqueda
    random_search.fit(X, y)
    
    optimization_time = time.time() - start_time
    
    print(f"     Optimización completada en {optimization_time:.2f}s")
    print(f"     Mejor F1-Score CV (optimizado): {random_search.best_score_:.4f}")
    print(f"     Parámetros optimizados: {len(random_search.best_params_)} parámetros encontrados")
    
    return {
        'modelo_optimizado': random_search.best_estimator_,
        'mejores_parametros': random_search.best_params_,
        'mejor_score_cv': random_search.best_score_,
        'tiempo_optimizacion': optimization_time,
        'search_object': random_search
    }

# Optimizar todos los modelos
print("OPTIMIZACION DE HIPERPARAMETROS (FASE 2)")
print("=" * 60)
print(f"Configuración: {N_ITER_SEARCH} iteraciones por modelo, {CV_FOLDS}-fold CV")
print(f"Métrica objetivo: F1-Score (balance precision/recall)")
print(f"Semilla reproducible: {RANDOM_SEED}")

modelos_optimizados = {}

for i, (nombre, config) in enumerate(modelos_avanzados.items(), 1):
    print(f"MODELO {i}/4: {nombre}")
    
    # Mostrar espacio de busqueda
    total_combinations = np.prod([len(v) for v in config['params'].values()])
    print(f"   Espacio de búsqueda: {total_combinations:,} combinaciones posibles")
    
    resultado_opt = optimizar_modelo(
        nombre, 
        config['model'], 
        config['params'], 
        X_train_final, 
        y_train_final
    )
    modelos_optimizados[nombre] = resultado_opt
    print()  # Línea en blanco para separación

# Resumen de optimización
print("COMPARACION: BASELINE vs OPTIMIZADO")
print("=" * 70)
print("BASELINE = Hiperparámetros por defecto (sin optimización)")
print("OPTIMIZADO = Hiperparámetros encontrados por RandomizedSearchCV")
print("=" * 70)

# Crear tabla comparativa baseline vs optimizado
print(f"{'Modelo':<15} {'Baseline F1':<12} {'Optimizado F1':<14} {'Mejora':<10} {'Tiempo':<10}")
print("-" * 70)

for nombre, resultado in modelos_optimizados.items():
    baseline_f1 = df_baseline[df_baseline['modelo'] == nombre]['f1_test_mean'].iloc[0]
    optimizado_f1 = resultado['mejor_score_cv']
    mejora = optimizado_f1 - baseline_f1
    tiempo = resultado['tiempo_optimizacion']
    
    # Indicador visual de mejora
    indicador = "📈" if mejora > 0 else "📉" if mejora < 0 else "➡️"
    
    print(f"{nombre:<15} {baseline_f1:<12.4f} {optimizado_f1:<14.4f} {mejora:+<10.4f} {tiempo:<10.1f}s {indicador}")

# Identificar modelo con mejor score de optimización
mejor_modelo_opt = max(modelos_optimizados.items(), 
                      key=lambda x: x[1]['mejor_score_cv'])

print(f"\nMEJOR MODELO OPTIMIZADO (CV): {mejor_modelo_opt[0]}")
print(f"   F1-Score CV (optimizado): {mejor_modelo_opt[1]['mejor_score_cv']:.4f}")

# Mostrar hiperparametros del mejor modelo
print(f"   Mejores hiperparámetros encontrados:")
for param, valor in mejor_modelo_opt[1]['mejores_parametros'].items():
    print(f"     • {param}: {valor}")

CONFIGURACION DE REPRODUCIBILIDAD:
   Semilla global: 42
   Iteraciones búsqueda: 50
   Folds CV optimización: 3
   Todas las ejecuciones futuras darán resultados idénticos

OPTIMIZACION DE HIPERPARAMETROS (FASE 2)
Configuración: 50 iteraciones por modelo, 3-fold CV
Métrica objetivo: F1-Score (balance precision/recall)
Semilla reproducible: 42
MODELO 1/4: XGBoost
   Espacio de búsqueda: 2,916 combinaciones posibles
   OPTIMIZANDO: XGBoost
   Búsqueda aleatoria con 50 iteraciones y 3-fold CV
     Optimización completada en 6.20s
     Mejor F1-Score CV (optimizado): 0.9245
     Parámetros optimizados: 7 parámetros encontrados

MODELO 2/4: LightGBM
   Espacio de búsqueda: 8,748 combinaciones posibles
   OPTIMIZANDO: LightGBM
   Búsqueda aleatoria con 50 iteraciones y 3-fold CV
     Optimización completada en 562.20s
     Mejor F1-Score CV (optimizado): 0.9254
     Parámetros optimizados: 8 parámetros encontrados

MODELO 3/4: RandomForest
   Espacio de búsqueda: 648 combinaciones posibles


## 6. Evaluación Final en Conjunto de Test

In [8]:
# Función para evaluación completa en test set
def evaluar_test_completo(modelo, X_train, X_test, y_train, y_test, nombre_modelo):
    """
    Evalúa modelo en conjunto de test con métricas completas
    IMPORTANTE: El modelo ya está optimizado, solo se entrena una vez más con datos completos
    """
    # Entrenar modelo con los datos de entrenamiento
    print(f"   Entrenando {nombre_modelo} (OPTIMIZADO) en datos completos...")
    start_time = time.time()
    modelo.fit(X_train, y_train)
    training_time = time.time() - start_time
    
    # Predicciones en test
    start_time = time.time()
    y_pred = modelo.predict(X_test)
    y_pred_proba = modelo.predict_proba(X_test)[:, 1]
    prediction_time = time.time() - start_time
    
    # Calcular métricas
    resultados = {
        'modelo': nombre_modelo,
        'accuracy': accuracy_score(y_test, y_pred),
        'precision': precision_score(y_test, y_pred),
        'recall': recall_score(y_test, y_pred),
        'f1': f1_score(y_test, y_pred),
        'roc_auc': roc_auc_score(y_test, y_pred_proba),
        'tiempo_entrenamiento': training_time,
        'tiempo_prediccion': prediction_time,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba,
        'modelo_entrenado': modelo  # IMPORTANTE: Guardar el modelo ya entrenado
    }
    
    print(f"     F1-Score: {resultados['f1']:.4f}, ROC-AUC: {resultados['roc_auc']:.4f}")
    print(f"     Tiempo entrenamiento: {resultados['tiempo_entrenamiento']:.3f}s")
    
    return resultados

# Evaluar modelos optimizados en test
print("EVALUACION FINAL EN CONJUNTO DE TEST (FASE 3)")
print("=" * 70)
print("IMPORTANTE: Los modelos usan hiperparámetros OPTIMIZADOS (encontrados en Fase 2)")
print("IMPORTANTE: Cada modelo se entrena con hiperparámetros ya optimizados")
print(f"Semilla global: {RANDOM_SEED}")
print("=" * 70)

resultados_test_final = []

for i, (nombre, config_opt) in enumerate(modelos_optimizados.items(), 1):
    print(f"EVALUANDO MODELO {i}/4: {nombre} (con hiperparámetros optimizados)")
    
    # Usar el modelo optimizado (NO reentrenado)
    modelo_optimizado = config_opt['modelo_optimizado']
    
    resultado = evaluar_test_completo(
        modelo_optimizado,
        X_train_final, X_test_final, y_train_final, y_test_final,
        nombre
    )
    
    resultados_test_final.append(resultado)

print()

# Convertir a DataFrame
df_test_final = pd.DataFrame(resultados_test_final)

# Mostrar tabla comparativa
print("RESULTADOS FINALES EN TEST SET:")
print("=" * 90)
print("MODELOS CON HIPERPARAMETROS OPTIMIZADOS - EVALUACION EN DATOS INDEPENDIENTES")
print("=" * 90)

# Seleccionar columnas para mostrar
cols_display = ['modelo', 'accuracy', 'precision', 'recall', 'f1', 'roc_auc', 
               'tiempo_entrenamiento', 'tiempo_prediccion']

df_display_final = df_test_final[cols_display].copy()

# Formatear números
for col in ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']:
    df_display_final[col] = df_display_final[col].apply(lambda x: f"{x:.4f}")

for col in ['tiempo_entrenamiento', 'tiempo_prediccion']:
    df_display_final[col] = df_display_final[col].apply(lambda x: f"{x:.3f}s")

print(df_display_final.to_string(index=False))

# Identificar mejor modelo final (UN SOLO CRITERIO)
mejor_modelo_final = df_test_final.loc[df_test_final['f1'].idxmax()]

print(f"\nMEJOR MODELO FINAL (por F1-Score en Test): {mejor_modelo_final['modelo']}")
print(f"   F1-Score: {mejor_modelo_final['f1']:.4f}")
print(f"   ROC-AUC: {mejor_modelo_final['roc_auc']:.4f}")
print(f"   Accuracy: {mejor_modelo_final['accuracy']:.4f}")
print(f"   Precision: {mejor_modelo_final['precision']:.4f}")
print(f"   Recall: {mejor_modelo_final['recall']:.4f}")

# Guardar el mejor modelo (YA ENTRENADO con hiperparametros optimizados)
mejor_nombre = mejor_modelo_final['modelo']
mejor_modelo_entrenado = mejor_modelo_final['modelo_entrenado']

# Crear directorio para modelos
import os
os.makedirs('../models', exist_ok=True)

# Guardar modelo completo
modelo_path = f'../models/mejor_modelo_{mejor_nombre.lower()}.pkl'
joblib.dump(mejor_modelo_entrenado, modelo_path)

# Guardar metadatos del modelo
metadatos = {
    'nombre_modelo': mejor_nombre,
    'hiperparametros': modelos_optimizados[mejor_nombre]['mejores_parametros'],
    'metricas_test': {
        'f1_score': mejor_modelo_final['f1'],
        'roc_auc': mejor_modelo_final['roc_auc'],
        'accuracy': mejor_modelo_final['accuracy'],
        'precision': mejor_modelo_final['precision'],
        'recall': mejor_modelo_final['recall']
    },
    'semilla_reproducibilidad': RANDOM_SEED,
    'fecha_entrenamiento': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'datos_entrenamiento': {
        'n_muestras': X_train_final.shape[0],
        'n_features': X_train_final.shape[1],
        'balance_clases': y_train_final.mean()
    }
}

metadatos_path = f'../models/metadatos_{mejor_nombre.lower()}.json'
import json
with open(metadatos_path, 'w') as f:
    json.dump(metadatos, f, indent=2)

print(f"\nMODELO GUARDADO:")
print(f"   Modelo: {modelo_path}")
print(f"   Metadatos: {metadatos_path}")
print(f"   Semilla: {RANDOM_SEED} (reproducibilidad garantizada)")

EVALUACION FINAL EN CONJUNTO DE TEST (FASE 3)
IMPORTANTE: Los modelos usan hiperparámetros OPTIMIZADOS (encontrados en Fase 2)
IMPORTANTE: Cada modelo se entrena con hiperparámetros ya optimizados
Semilla global: 42
EVALUANDO MODELO 1/4: XGBoost (con hiperparámetros optimizados)
   Entrenando XGBoost (OPTIMIZADO) en datos completos...
     F1-Score: 0.6524, ROC-AUC: 0.9236
     Tiempo entrenamiento: 0.210s
EVALUANDO MODELO 2/4: LightGBM (con hiperparámetros optimizados)
   Entrenando LightGBM (OPTIMIZADO) en datos completos...
     F1-Score: 0.6509, ROC-AUC: 0.9218
     Tiempo entrenamiento: 0.110s
EVALUANDO MODELO 3/4: RandomForest (con hiperparámetros optimizados)
   Entrenando RandomForest (OPTIMIZADO) en datos completos...
     F1-Score: 0.6542, ROC-AUC: 0.9198
     Tiempo entrenamiento: 0.488s
EVALUANDO MODELO 4/4: GradientBoosting (con hiperparámetros optimizados)
   Entrenando GradientBoosting (OPTIMIZADO) en datos completos...
     F1-Score: 0.6489, ROC-AUC: 0.9236
     Tiempo 

## 7. Definición de Variables y Análisis de Feature Importance

### 7.1 Definición de Variables

In [9]:
target_variable = 'G_11_F'  # Consumo de marihuana (8% prevalencia)

# Variables predictoras categóricas (entorno social + actitudes)
categorical_features = [
    # Entorno Social
    'G_01',  # Familiares consumen sustancias
    'G_02',  # Amigos consumen sustancias 
    
    # Actitudes y Curiosidad
    'G_03',  # Curiosidad por probar
    'G_04',  # Disposición a consumir
    'G_05'   # Tuvo oportunidad de probar
]

# Variables predictoras numéricas (accesibilidad + exposición)
numerical_features = [
    # Cantidad de familiares/amigos
    'G_01_A',  # Cantidad de familiares que consumen
    'G_02_A',  # Cantidad de amigos que consumen
    
    # Accesibilidad (facilidad de acceso)
    'G_06_A',  # Facilidad acceso marihuana
    'G_06_B',  # Facilidad acceso cocaína
    'G_06_C',  # Facilidad acceso basuco
    'G_06_D',  # Facilidad acceso éxtasis
    
    # Exposición (ofertas recibidas)
    'G_07',    # Ofertas recibidas en último año
    'G_08_A',  # Ofertas de marihuana
    'G_08_B'   # Ofertas de cocaína
]

# Diccionario de nombres legibles
variable_names = {
    # Variable objetivo
    'G_11_F': 'Consumo de Marihuana',
    
    # Variables categóricas - Entorno Social
    'G_01': 'Familiares Consumen Sustancias',
    'G_02': 'Amigos Consumen Sustancias',
    
    # Variables categóricas - Actitudes
    'G_03': 'Curiosidad por Probar',
    'G_04': 'Disposición a Consumir',
    'G_05': 'Tuvo Oportunidad de Probar',
    
    # Variables numéricas - Cantidades
    'G_01_A': 'Cantidad de Familiares que Consumen',
    'G_02_A': 'Cantidad de Amigos que Consumen',
    
    # Variables numéricas - Accesibilidad
    'G_06_A': 'Acceso Fácil a Marihuana',
    'G_06_B': 'Acceso Fácil a Cocaína',
    'G_06_C': 'Acceso Fácil a Basuco',
    'G_06_D': 'Acceso Fácil a Éxtasis',
    
    # Variables numéricas - Exposición
    'G_07': 'Ofertas Recibidas (Último Año)',
    'G_08_A': 'Ofertas de Marihuana',
    'G_08_B': 'Ofertas de Cocaína'
}

# Mapeo adicional para variables codificadas (one-hot encoding)
# Estas aparecen en los resultados de feature importance
encoded_variable_names = {
    # Variables categóricas codificadas (valor 1.0 = "Sí")
    'G_01_1.0': 'Familiares Consumen Sustancias (Sí)',
    'G_02_1.0': 'Amigos Consumen Sustancias (Sí)',
    'G_03_1.0': 'Curiosidad por Probar (Sí)',
    'G_04_1.0': 'Disposición a Consumir (Sí)',
    'G_05_1.0': 'Tuvo Oportunidad de Probar (Sí)',
    
    # Variables categóricas codificadas (valor 0.0 = "No")
    'G_01_0.0': 'Familiares Consumen Sustancias (No)',
    'G_02_0.0': 'Amigos Consumen Sustancias (No)',
    'G_03_0.0': 'Curiosidad por Probar (No)',
    'G_04_0.0': 'Disposición a Consumir (No)',
    'G_05_0.0': 'Tuvo Oportunidad de Probar (No)'
}

# Combinar ambos diccionarios
all_variable_names = {**variable_names, **encoded_variable_names}

# Función para obtener nombre legible (consistente con notebooks anteriores)
def get_readable_name(var_code):
    """Convierte código de variable a nombre legible"""
    return all_variable_names.get(var_code, var_code)

# Todas las variables para el análisis
all_features = categorical_features + numerical_features + [target_variable]

### 7.2 Análisis de Feature Importance

In [10]:
# Función para extraer feature importance NORMALIZADA
def extraer_feature_importance(modelo_entrenado, feature_names, nombre_modelo):
    """
    Extrae feature importance de modelo YA ENTRENADO con normalización
    """
    try:
        if hasattr(modelo_entrenado, 'feature_importances_'):
            importances = modelo_entrenado.feature_importances_
        elif hasattr(modelo_entrenado, 'coef_'):
            importances = np.abs(modelo_entrenado.coef_[0])
        else:
            return None
        
        # NORMALIZAR importancias para hacer comparables entre modelos
        # Método: Normalización por suma (cada modelo suma 1.0)
        importances_normalized = importances / importances.sum()
        
        # Crear DataFrame
        df_importance = pd.DataFrame({
            'feature': feature_names,
            'importance': importances_normalized,  # Usar importancias normalizadas
            'importance_raw': importances,         # Mantener valores originales
            'modelo': nombre_modelo
        }).sort_values('importance', ascending=False)
        
        return df_importance
        
    except Exception as e:
        print(f"Error extrayendo importances para {nombre_modelo}: {e}")
        return None

# Extraer importances de modelos YA ENTRENADOS (sin reentrenar)
feature_names = X_train_final.columns.tolist()
all_importances = []

print("ANALISIS DE FEATURE IMPORTANCE NORMALIZADA")
print("=" * 60)
print("Usando modelos ya entrenados con hiperparametros optimizados")
print("IMPORTANTE: Importancias normalizadas para comparación entre modelos")
print("Cada modelo suma 1.0 - valores comparables entre algoritmos")
print("=" * 60)

for resultado in resultados_test_final:
    nombre = resultado['modelo']
    modelo_entrenado = resultado['modelo_entrenado']
    
    df_imp = extraer_feature_importance(modelo_entrenado, feature_names, nombre)
    
    if df_imp is not None:
        all_importances.append(df_imp)
        print(f"{nombre}: {len(df_imp)} features extraidas")
    else:
        print(f"{nombre}: No se pudo extraer feature importance")

# Combinar todas las importances
if all_importances:
    df_all_importances = pd.concat(all_importances, ignore_index=True)
    
    # Verificar normalización
    print(f"\nVERIFICACION DE NORMALIZACION:")
    for modelo in df_all_importances['modelo'].unique():
        suma_modelo = df_all_importances[df_all_importances['modelo'] == modelo]['importance'].sum()
        print(f"   {modelo:15s}: Suma = {suma_modelo:.4f} (debe ser ≈ 1.0)")
    
    # Tabla pivot para comparación (importancias normalizadas)
    df_importance_pivot = df_all_importances.pivot(
        index='feature', columns='modelo', values='importance'
    ).fillna(0)
    
    # Tabla pivot para valores sin normalizar (para referencia)
    df_importance_raw = df_all_importances.pivot(
        index='feature', columns='modelo', values='importance_raw'
    ).fillna(0)
    
    # Calcular importancia promedio
    df_importance_pivot['promedio'] = df_importance_pivot.mean(axis=1)
    df_importance_pivot = df_importance_pivot.sort_values('promedio', ascending=False)
    
    print(f"\nTOP 10 FEATURES MAS IMPORTANTES (Importancias Normalizadas):")
    print("=" * 70)
    print("Valores normalizados - comparables entre todos los modelos")
    
    # Mostrar top 10
    top_features = df_importance_pivot.head(10)
    
    for idx, (feature, row) in enumerate(top_features.iterrows(), 1):
        pretty_feature = get_readable_name(feature)
        print(f"{idx:2d}. {pretty_feature:20s} | Promedio: {row['promedio']:.4f}")
        
        # Mostrar por modelo
        for modelo in [r['modelo'] for r in resultados_test_final]:
            if modelo in row.index and row[modelo] > 0:
                print(f"     {modelo:15s}: {row[modelo]:.4f}")
        print()
    
    # Crear visualización
    fig = go.Figure()
    
    # Agregar barras para cada modelo
    colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
    x_labels = [get_readable_name(f) for f in top_features.index]
    for i, modelo in enumerate([r['modelo'] for r in resultados_test_final]):
        if modelo in df_importance_pivot.columns:
            fig.add_trace(go.Bar(
                name=modelo,
                x=x_labels,
                y=top_features[modelo],
                text=[f'{v:.3f}' for v in top_features[modelo]],
                textposition='auto',
                marker_color=colores[i % len(colores)]
            ))
    
    fig.update_layout(
        title='Feature Importance Normalizada por Modelo - Top 10 Features<br><sub>Valores normalizados (suma=1.0) - Comparables entre algoritmos</sub>',
        xaxis_title='Features',
        yaxis_title='Importance Score (Normalizado)',
        barmode='group',
        height=600,
        showlegend=True,
        xaxis={'tickangle': 45}
    )
    
    fig.show()
    
    # Guardar feature importance del mejor modelo
    mejor_modelo_name = mejor_modelo_final['modelo']
    if mejor_modelo_name in df_importance_pivot.columns:
        top_features_mejor = df_importance_pivot[mejor_modelo_name].sort_values(ascending=False).head(15)
        
        print(f"\nTOP 15 FEATURES DEL MEJOR MODELO ({mejor_modelo_name}) - NORMALIZADAS:")
        print("=" * 65)
        print("Importancias normalizadas (suma total = 1.0)")
        print("-" * 40)
        for idx, (feature, importance) in enumerate(top_features_mejor.items(), 1):
            pretty_feature = get_readable_name(feature)
            print(f"{idx:2d}. {pretty_feature:20s}: {importance:.4f}")
    
else:
    print("No se pudieron extraer feature importances de ningun modelo")

ANALISIS DE FEATURE IMPORTANCE NORMALIZADA
Usando modelos ya entrenados con hiperparametros optimizados
IMPORTANTE: Importancias normalizadas para comparación entre modelos
Cada modelo suma 1.0 - valores comparables entre algoritmos
XGBoost: 14 features extraidas
LightGBM: 14 features extraidas
RandomForest: 14 features extraidas
GradientBoosting: 14 features extraidas

VERIFICACION DE NORMALIZACION:
   XGBoost        : Suma = 1.0000 (debe ser ≈ 1.0)
   LightGBM       : Suma = 1.0000 (debe ser ≈ 1.0)
   RandomForest   : Suma = 1.0000 (debe ser ≈ 1.0)
   GradientBoosting: Suma = 1.0000 (debe ser ≈ 1.0)

TOP 10 FEATURES MAS IMPORTANTES (Importancias Normalizadas):
Valores normalizados - comparables entre todos los modelos
 1. Curiosidad por Probar (Sí) | Promedio: 0.4882
     XGBoost        : 0.6989
     LightGBM       : 0.0452
     RandomForest   : 0.4360
     GradientBoosting: 0.7728

 2. Tuvo Oportunidad de Probar (Sí) | Promedio: 0.0817
     XGBoost        : 0.0643
     LightGBM     


TOP 15 FEATURES DEL MEJOR MODELO (RandomForest) - NORMALIZADAS:
Importancias normalizadas (suma total = 1.0)
----------------------------------------
 1. Curiosidad por Probar (Sí): 0.4360
 2. Tuvo Oportunidad de Probar (Sí): 0.1545
 3. Disposición a Consumir (Sí): 0.1069
 4. Ofertas Recibidas (Último Año): 0.0520
 5. Ofertas de Marihuana: 0.0515
 6. Acceso Fácil a Éxtasis: 0.0369
 7. Acceso Fácil a Marihuana: 0.0344
 8. Acceso Fácil a Basuco: 0.0292
 9. Acceso Fácil a Cocaína: 0.0282
10. Ofertas de Cocaína  : 0.0241
11. Amigos Consumen Sustancias (Sí): 0.0231
12. Familiares Consumen Sustancias (Sí): 0.0126
13. Cantidad de Familiares que Consumen: 0.0055
14. Cantidad de Amigos que Consumen: 0.0051


## 8. Curvas ROC Comparativas

In [11]:
# Crear curvas ROC para todos los modelos
fig_roc = go.Figure()

print("Generando Curvas ROC Comparativas...\n")

# Colores para los modelos
colores = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

roc_data = []

for i, resultado in enumerate(resultados_test_final):
    nombre = resultado['modelo']
    y_pred_proba = resultado['y_pred_proba']
    
    # Calcular curva ROC
    fpr, tpr, thresholds = roc_curve(y_test_final, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # Guardar datos
    roc_data.append({
        'modelo': nombre,
        'fpr': fpr,
        'tpr': tpr,
        'auc': roc_auc
    })
    
    # Agregar curva al gráfico
    color = colores[i % len(colores)]
    
    fig_roc.add_trace(go.Scatter(
        x=fpr,
        y=tpr,
        mode='lines',
        name=f'{nombre} (AUC = {roc_auc:.3f})',
        line=dict(color=color, width=2)
    ))
    
    print(f"{nombre}: AUC = {roc_auc:.4f}")

# Agregar línea diagonal (clasificador aleatorio)
fig_roc.add_trace(go.Scatter(
    x=[0, 1],
    y=[0, 1],
    mode='lines',
    name='Clasificador Aleatorio (AUC = 0.500)',
    line=dict(color='gray', width=1, dash='dash')
))

# Configurar layout
fig_roc.update_layout(
    title='Curvas ROC - Comparación de Modelos Avanzados',
    xaxis_title='Tasa de Falsos Positivos (1 - Especificidad)',
    yaxis_title='Tasa de Verdaderos Positivos (Sensibilidad)',
    width=800,
    height=600,
    showlegend=True,
    xaxis=dict(range=[0, 1]),
    yaxis=dict(range=[0, 1])
)

fig_roc.show()

# Tabla resumen de AUC
print("\nResumen de AUC Scores:")
print("=" * 30)

roc_summary = sorted(roc_data, key=lambda x: x['auc'], reverse=True)

for i, data in enumerate(roc_summary, 1):
    print(f"{i}. {data['modelo']:15s}: {data['auc']:.4f}")

# Diferencia con el mejor
mejor_auc = roc_summary[0]['auc']
print(f"\nDiferencias respecto al mejor modelo:")
for data in roc_summary[1:]:
    diff = mejor_auc - data['auc']
    print(f"   {data['modelo']:15s}: -{diff:.4f} AUC")

Generando Curvas ROC Comparativas...

XGBoost: AUC = 0.9236
LightGBM: AUC = 0.9218
RandomForest: AUC = 0.9198
GradientBoosting: AUC = 0.9236



Resumen de AUC Scores:
1. XGBoost        : 0.9236
2. GradientBoosting: 0.9236
3. LightGBM       : 0.9218
4. RandomForest   : 0.9198

Diferencias respecto al mejor modelo:
   GradientBoosting: -0.0000 AUC
   LightGBM       : -0.0018 AUC
   RandomForest   : -0.0038 AUC


## 9. Matriz de Confusión del Mejor Modelo

In [12]:
# Obtener el mejor modelo y sus predicciones
mejor_resultado = df_test_final.loc[df_test_final['f1'].idxmax()]
mejor_nombre = mejor_resultado['modelo']
mejor_y_pred = mejor_resultado['y_pred']

# Calcular matriz de confusión
cm = confusion_matrix(y_test_final, mejor_y_pred)

# Crear heatmap interactivo
fig_cm = go.Figure(data=go.Heatmap(
    z=cm,
    x=['Predicho: No Consume', 'Predicho: Sí Consume'],
    y=['Real: No Consume', 'Real: Sí Consume'],
    colorscale='Blues',
    text=cm,
    texttemplate="%{text}",
    textfont={"size": 16},
    hoverongaps=False
))

fig_cm.update_layout(
    title=f'Matriz de Confusión - {mejor_nombre}',
    xaxis_title='Predicción',
    yaxis_title='Valor Real',
    width=500,
    height=400
)

fig_cm.show()

# Calcular métricas detalladas de la matriz de confusión
tn, fp, fn, tp = cm.ravel()

print(f"\nAnálisis Detallado - {mejor_nombre}")
print("=" * 50)
print(f"Matriz de Confusión:")
print(f"   Verdaderos Negativos (TN): {tn:,}")
print(f"   Falsos Positivos (FP):     {fp:,}")
print(f"   Falsos Negativos (FN):     {fn:,}")
print(f"   Verdaderos Positivos (TP): {tp:,}")

print(f"\nMétricas Calculadas:")
print(f"   Accuracy:    {(tp + tn) / (tp + tn + fp + fn):.4f}")
print(f"   Precision:   {tp / (tp + fp):.4f}")
print(f"   Recall:      {tp / (tp + fn):.4f}")
print(f"   Specificity: {tn / (tn + fp):.4f}")
print(f"   F1-Score:    {2 * tp / (2 * tp + fp + fn):.4f}")

# Interpretación de resultados
print(f"\nInterpretación de Resultados:")
print(f"   • De {tn + fp:,} personas que NO consumen, el modelo identifica correctamente {tn:,} ({tn/(tn+fp):.1%})")
print(f"   • De {tp + fn:,} personas que SÍ consumen, el modelo identifica correctamente {tp:,} ({tp/(tp+fn):.1%})")
print(f"   • Falsos positivos: {fp:,} personas ({fp/(tn+fp):.1%} de los no consumidores)")
print(f"   • Falsos negativos: {fn:,} personas ({fn/(tp+fn):.1%} de los consumidores)")

# Reporte de clasificación completo
print(f"\nReporte de Clasificación Completo:")
print("=" * 60)
print(classification_report(y_test_final, mejor_y_pred, 
                          target_names=['No Consume', 'Sí Consume']))


Análisis Detallado - RandomForest
Matriz de Confusión:
   Verdaderos Negativos (TN): 3,534
   Falsos Positivos (FP):     397
   Falsos Negativos (FN):     195
   Verdaderos Positivos (TP): 560

Métricas Calculadas:
   Accuracy:    0.8737
   Precision:   0.5852
   Recall:      0.7417
   Specificity: 0.8990
   F1-Score:    0.6542

Interpretación de Resultados:
   • De 3,931 personas que NO consumen, el modelo identifica correctamente 3,534 (89.9%)
   • De 755 personas que SÍ consumen, el modelo identifica correctamente 560 (74.2%)
   • Falsos positivos: 397 personas (10.1% de los no consumidores)
   • Falsos negativos: 195 personas (25.8% de los consumidores)

Reporte de Clasificación Completo:
              precision    recall  f1-score   support

  No Consume       0.95      0.90      0.92      3931
  Sí Consume       0.59      0.74      0.65       755

    accuracy                           0.87      4686
   macro avg       0.77      0.82      0.79      4686
weighted avg       0.89  